In [1]:
%matplotlib notebook
import sys
from bingo.evolutionary_optimizers.parallel_archipelago \
    import load_parallel_archipelago_from_file
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from ipywidgets import interact
import os
import gc
import h5py
import pprint


In [23]:
directories = ["fs1d29e10_even/pkl_files/"]

def get_labels(filename):
    label_map = {"pql": ["Sig_h", "Sig_vm", "Lode", "vvf"],
                 "haigh_westergaard": ["z", "r", "theta", "eps", "vvf"],
                 "invariants": ["I1", "J2", "J3", "eps", "vvf"],
                 "principle_stresses": ["Sig_1", "Sig_2", "Sig_3", "eps", "vvf"],
                 "custom": ["Sig_h", "Sig_vm", "vvf"], # use for runs before July
                 "lode": ["Sig_h","Sig_vm","Lode","vvf"],
                 "no_lode": ["Sig_h", "Sig_vm", "vvf"],
                  "porosity": ["vvf", "del_vvf", "del_PE11", "del_PE22"]}
    for key, label in label_map.items():
        if key in filename:
            return label
    return None
        
data_label_map = {}
for d in directories:
    for f in os.listdir(d):
        if f.endswith(".pkl"):
            labels = get_labels(f)
            data_label_map[os.path.join(d, f)] = labels

hof_index_list = [i for i in range(-10, 10)]
filename_list = list(data_label_map.keys())

In [ ]:
print(hof_index_list)
print(filename_list)
# print(data_label_map)

In [ ]:
def print_eq_and_fitness(filename, hof_index=0):
    archipelago = load_parallel_archipelago_from_file(filename)
    hof = archipelago.hall_of_fame
    equation = hof[hof_index]
#     equation2 = equation.copy()
#     equation2._use_simplification = True
#     equation2._update()

    print('Equation:', equation)
#     print('Simplified Equation:', equation2)
    print('Fitness:', hof[hof_index].fitness)
    print('Complexity:', hof[hof_index].get_complexity())
    allhofs = np.arange(-10,10,1)
    allfitness = []
    for i in allhofs:
        allfitness.append(hof[i].fitness)
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.plot(allhofs,allfitness)
#     ax.set_ylabel('Fitness')
#     ax.set_xlabel('Hof Index')
#     fig.show()
    
interact(print_eq_and_fitness, filename=filename_list, hof_index=hof_index_list)

In [ ]:
def plot_pareto(filename, hof_index=0):
    archipelago = load_parallel_archipelago_from_file(filename)
    hof = archipelago.hall_of_fame
    equation = hof[hof_index]
    
    print('Equation:', equation)
    print('Fitness:', hof[hof_index].fitness)
    print('Complexity:', hof[hof_index].get_complexity())
    allhofs = np.arange(-10,10,1)
    hofspos = np.arange(0,11)
    hofsneg = np.arange(-10,0)
    allhofs = np.concatenate((hofspos, hofsneg), axis=None)
    allfitness = []
    allcomplexity = []
    for i in allhofs:
        allfitness.append(hof[i].fitness)
        allcomplexity.append(hof[i].get_complexity())
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.step(allcomplexity,allfitness,'k',where='post')
    ax.plot(allcomplexity,allfitness,'ro')
    ax.set_ylabel('Fitness')
    ax.set_xlabel('Complexity')
    fig.show()
    
interact(plot_pareto, filename=filename_list, hof_index=hof_index_list)

In [24]:
def _get_data_and_labels(filename, hof_index=None, fit_func=False):
    archipelago = load_parallel_archipelago_from_file(filename)
    data = archipelago.island._full_training_data
#     data = archipelago._island._ea.evaluation.fitness_function.training_data 
    data_labels = data_label_map[filename]
    output = (data, data_labels)
    if hof_index is not None:
        hof = archipelago.hall_of_fame
        output += (hof[hof_index],)
    
    if fit_func:
        ff = archipelago.island._fitness_function
        output += (ff,)
        
    return output

In [ ]:
# this will plot the mean of the function evaluated at the data
def mean_data(filename, hof_index=0):
    
    data, data_labels, equation = _get_data_and_labels(filename, hof_index)
    equation_2 = equation.copy()
#     equation_2._use_simplification = False
    print(equation_2)
    data = data.x
    print(data)
    print(np.shape(data))
    equ_eval = equation.evaluate_equation_at(data) # evaluate the equation at the data (if the equation was Gurson, this would give zero at all data points)
    mean_const = np.mean(equ_eval) 
    print(mean_const)
 

interact(mean_data, filename=filename_list, hof_index=hof_index_list)

In [ ]:
def plot_data(filename, x_ind=0, y_ind=1):
    data, data_labels = _get_data_and_labels(filename)
    print(data.x.shape)
    data_labels = ['Sig_h', 'Sig_vm']
    
    fig = plt.figure()
    plt.plot(data.x[:, x_ind], data.x[:, y_ind])    
    
interact(plot_data, filename=filename_list)

In [ ]:
def plot_data(filename, x_ind=0, y_ind=1, z_ind=2, c_ind=3):
    data, data_labels = _get_data_and_labels(filename)
    print(data.x.shape)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(data.x[:, x_ind], data.x[:, y_ind], data.x[:, z_ind], c=data.x[:, c_ind])
    ax.set_xlabel(data_labels[x_ind])
    ax.set_ylabel(data_labels[y_ind])
    ax.set_zlabel(data_labels[z_ind])
    fig.colorbar(sc)
    
interact(plot_data, filename=filename_list, x_ind=range(5), y_ind=range(5), z_ind=range(5), c_ind=range(5))

In [ ]:
def plot_data_and_potential_field(filename, hof_index=0, x_ind=0, y_ind=1, z_ind=3):
    
    data, data_labels, equation = _get_data_and_labels(filename, hof_index)
    equation_2 = equation.copy()
    equation_2._use_simplification = False
#     equation_2._update()
    print(equation_2)
    data = data.x
    mean_data = np.mean(data, axis=0) # gets the mean hydrostatic, von Mises, Lode, vvf
    syield = 1 # this assumes the yield stress is normalized! make sure I always normalize in run_bingo.py
    equ_eval = equation.evaluate_equation_at(data) # evaluate the equation at the data (if the equation was Gurson, this would give zero at all data points)
    mean_const = np.mean(equ_eval) 
    std_const = np.std(equ_eval)
    gurson = (data[:,1]/syield)**2+2*data[:,3]*np.cosh(1.5*(data[:,0]/syield))-(1+data[:,3]**2)
    mean_gurson = np.mean(gurson)
#     rel_per_diff = np.mean(np.absolute((gurson-equ_eval))) # THIS CAUSES THE KERNEL TO CRASH
#     print(rel_per_diff)
    npts = 50
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x_vec = np.linspace(np.min(data[:,x_ind]), np.max(data[:,x_ind]), npts)
    y_vec = np.linspace(np.min(data[:,y_ind]), np.max(data[:,y_ind]), npts)
    z_vec = np.linspace(np.min(data[:,z_ind]), np.max(data[:,z_ind]), npts)
    x, y = np.meshgrid(x_vec, y_vec) # so here he's just got a meshgrid of all the von Mises and hydrostatic stresses from his data

#     #levels = np.linspace(mean_const -2*std_const, mean_const + 2*std_const, 7)
    levels = [mean_const]             # these levels define what part of the contours he wants to show up, i.e. just pretty much 0
    levels_gurson = [mean_gurson]
#     levels = [0]
#     levels_gurson = [0]
    print(levels)
    print(levels_gurson)
    
    for z in z_vec: # for each porosity he's doing this 
        flat_data = np.vstack([np.full_like(x.flatten(), m) for m in mean_data]).T
        flat_data[:, x_ind] = x.flatten()
        flat_data[:, y_ind] = y.flatten()
        flat_data[:, z_ind] = z 
        F = equation.evaluate_equation_at(flat_data).reshape(x.shape)
        F_gurson = (flat_data[:,1]/syield)**2+2*flat_data[:,3]*np.cosh(1.5*(flat_data[:,0]/syield))-(1+flat_data[:,3]**2)
        F_gurson = F_gurson.reshape(x.shape)
        # so here's what's happening: the "F" from the Gurson evaluation is what the z part of his contour is. 
        # then, with levels, he only includes the contours at THAT level, i.e. at 0 where those points satisfy
        # being on the Gurson yield surface. Now, he has the curves at 0, and simply just moves them up to where 
        # they should be. I think this looks pretty legit, can't really see anything wrong with it. 
        # PUT SHORTLY: he loops through different vvf's present in the data; using those vvfs, gets the curve 
        # through all the stresses that satisfy Gurson. 
        # ONE question I guess: Why is he defining the levels based off of the mean value of the Gurson yield function
        # on his data? Why not just at 0 where the surface would be? 
        cset = ax.contour(x, y, F, levels, offset=z,colors='red')
        cset_gurson = ax.contour(x, y, F_gurson, levels_gurson, offset=z,colors='blue')
            
#     sc = ax.scatter(data[::50, x_ind], data[::50, y_ind], data[::50, z_ind], c="k")
    sc = ax.scatter(data[:, x_ind], data[:, y_ind], data[:, z_ind], c="k")
    ax.set_xlabel(data_labels[x_ind])
    ax.set_ylabel(data_labels[y_ind])
    ax.set_zlabel(data_labels[z_ind])
    ax.set_xlim(x_vec[0], x_vec[-1])
    ax.set_ylim(y_vec[0], y_vec[-1])
    ax.set_zlim(z_vec[0], z_vec[-1])
#     fig.colorbar(cset)
    plt.show()
    
#     threshold = 0.25
#     F_gurson = F_gurson.flatten()
#     F = F.flatten()
#     rel_perc_diff = np.zeros(len(F_gurson))
#     for i in range(0,len(F_gurson)):
#         if F_gurson[i] <= threshold:
#             rel_perc_diff[i] = rel_perc_diff[i-1]
#         else:
#             rel_perc_diff[i] = np.absolute((F_gurson[i]-F[i])/F_gurson[i]*100)
#     print(np.mean(rel_perc_diff))

interact(plot_data_and_potential_field, filename=filename_list, hof_index=hof_index_list,
         x_ind=range(5), y_ind=range(5), z_ind=range(5))

In [ ]:
def plot_data_and_potential_field_noLode(filename, hof_index=0, x_ind=0, y_ind=1, z_ind=2):
    
    data, data_labels, equation = _get_data_and_labels(filename, hof_index)
    equation_2 = equation.copy()
    equation_2._use_simplification = False
#     equation_2._update()
    print(equation_2)
    data = data.x
    mean_data = np.mean(data, axis=0) # gets the mean hydrostatic, von Mises, vvf
    syield = 1 # this assumes the yield stress is normalized! make sure I always normalize in run_bingo.py
    equ_eval = equation.evaluate_equation_at(data) # evaluate the equation at the data (if the equation was Gurson, this would give zero at all data points)
    mean_const = np.mean(equ_eval) 
    std_const = np.std(equ_eval)
    gurson = (data[:,1]/syield)**2+2*data[:,2]*np.cosh(1.5*(data[:,0]/syield))-(1+data[:,2]**2)
    mean_gurson = np.mean(gurson)
#     rel_per_diff = np.mean(np.absolute((gurson-equ_eval))) # THIS CAUSES THE KERNEL TO CRASH
#     print(rel_per_diff)
    npts = 50
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x_vec = np.linspace(np.min(data[:,x_ind]), np.max(data[:,x_ind]), npts)
    y_vec = np.linspace(np.min(data[:,y_ind]), np.max(data[:,y_ind]), npts)
    z_vec = np.linspace(np.min(data[:,z_ind]), np.max(data[:,z_ind]), npts)
    x, y = np.meshgrid(x_vec, y_vec) # so here he's just got a meshgrid of all the von Mises and hydrostatic stresses from his data

#     #levels = np.linspace(mean_const -2*std_const, mean_const + 2*std_const, 7)
    levels = [mean_const]             # these levels define what part of the contours he wants to show up, i.e. just pretty much 0
    levels_gurson = [mean_gurson]
#     levels = [0]
#     levels_gurson = [0]
    print(levels)
    print(levels_gurson)
    
    for z in z_vec: # for each porosity he's doing this 
        flat_data = np.vstack([np.full_like(x.flatten(), m) for m in mean_data]).T
        flat_data[:, x_ind] = x.flatten()
        flat_data[:, y_ind] = y.flatten()
        flat_data[:, z_ind] = z 
        F = equation.evaluate_equation_at(flat_data).reshape(x.shape)
        F_gurson = (flat_data[:,1]/syield)**2+2*flat_data[:,2]*np.cosh(1.5*(flat_data[:,0]/syield))-(1+flat_data[:,2]**2)
        F_gurson = F_gurson.reshape(x.shape)
        # so here's what's happening: the "F" from the Gurson evaluation is what the z part of his contour is. 
        # then, with levels, he only includes the contours at THAT level, i.e. at 0 where those points satisfy
        # being on the Gurson yield surface. Now, he has the curves at 0, and simply just moves them up to where 
        # they should be. I think this looks pretty legit, can't really see anything wrong with it. 
        # PUT SHORTLY: he loops through different vvf's present in the data; using those vvfs, gets the curve 
        # through all the stresses that satisfy Gurson. 
        # ONE question I guess: Why is he defining the levels based off of the mean value of the Gurson yield function
        # on his data? Why not just at 0 where the surface would be? 
        cset = ax.contour(x, y, F, levels, offset=z,colors='red')
        cset_gurson = ax.contour(x, y, F_gurson, levels_gurson, offset=z,colors='blue')
            
#     sc = ax.scatter(data[::50, x_ind], data[::50, y_ind], data[::50, z_ind], c="k")
    sc = ax.scatter(data[:, x_ind], data[:, y_ind], data[:, z_ind], c="k")
    ax.set_xlabel(data_labels[x_ind])
    ax.set_ylabel(data_labels[y_ind])
    ax.set_zlabel(data_labels[z_ind])
    ax.set_xlim(x_vec[0], x_vec[-1])
    ax.set_ylim(y_vec[0], y_vec[-1])
    ax.set_zlim(z_vec[0], z_vec[-1])
#     fig.colorbar(cset)
    plt.show()
    
#     threshold = 0.25
#     F_gurson = F_gurson.flatten()
#     F = F.flatten()
#     rel_perc_diff = np.zeros(len(F_gurson))
#     for i in range(0,len(F_gurson)):
#         if F_gurson[i] <= threshold:
#             rel_perc_diff[i] = rel_perc_diff[i-1]
#         else:
#             rel_perc_diff[i] = np.absolute((F_gurson[i]-F[i])/F_gurson[i]*100)
#     print(np.mean(rel_perc_diff))

interact(plot_data_and_potential_field_noLode, filename=filename_list, hof_index=hof_index_list,
         x_ind=range(5), y_ind=range(5), z_ind=range(5))

In [ ]:
def plot_data_and_potential_field_onlyyields(filename, hof_index=0, x_ind=0, y_ind=1, z_ind=2):
    
    data, data_labels, equation = _get_data_and_labels(filename, hof_index)
    equation_2 = equation.copy()
    equation_2._use_simplification = False
#     equation_2._update()
    print(equation_2)
    data = data.x
    mean_data = np.mean(data, axis=0)
    syield = 1
    
    equ_eval = equation.evaluate_equation_at(data)
    mean_const = np.mean(equ_eval)
    std_const = np.std(equ_eval)
    gurson = (data[:,1]/syield)**2+2*data[:,2]*np.cosh(1.5*(data[:,0]/syield))-(1+data[:,2]**2)
    mean_gurson = np.mean(gurson) # gets the mean value of phi that Gurson evaluates to w/ the data? 
    rel_per_diff = np.mean(np.absolute((gurson-equ_eval)))
#     print(rel_per_diff)
    npts = 50

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x_vec = np.linspace(np.min(data[:,x_ind]), np.max(data[:,x_ind]), npts) 
    y_vec = np.linspace(np.min(data[:,y_ind]), np.max(data[:,y_ind]), npts)
    z_vec = np.linspace(np.min(data[:,z_ind]), np.max(data[:,z_ind]), npts)
    x, y = np.meshgrid(x_vec, y_vec) # gets a meshgrid of von Mises and hydrostatic stresses between mins and maxes in his data

#     #levels = np.linspace(mean_const -2*std_const, mean_const + 2*std_const, 7)
    levels = [mean_const]
    levels_gurson = [mean_gurson]
    
    
    for z in z_vec:
        flat_data = np.vstack([np.full_like(x.flatten(), m) for m in mean_data]).T
        flat_data[:, x_ind] = x.flatten()
        flat_data[:, y_ind] = y.flatten()
        flat_data[:, z_ind] = z
        F = equation.evaluate_equation_at(flat_data).reshape(x.shape)
        F_gurson = (flat_data[:,1]/syield)**2+2*flat_data[:,2]*np.cosh(1.5*(flat_data[:,0]/syield))-(1+flat_data[:,2]**2)
        F_gurson = F_gurson.reshape(x.shape) 
        cset = ax.contour(x, y, F, levels, offset=z,colors='red')
        cset_gurson = ax.contour(x, y, F_gurson, levels_gurson, offset=z,colors='blue')
            
    sc = ax.scatter(data[:, x_ind], data[:, y_ind], data[:, z_ind], c="k")
    ax.set_xlabel(data_labels[x_ind])
    ax.set_ylabel(data_labels[y_ind])
    ax.set_zlabel(data_labels[z_ind])
    ax.set_xlim(x_vec[0], x_vec[-1])
    ax.set_ylim(y_vec[0], y_vec[-1])
    ax.set_zlim(z_vec[0], z_vec[-1])
    #fig.colorbar(cset)
    
    plt.show()
    
    threshold = 0.25
    F_gurson = F_gurson.flatten()
    F = F.flatten()
    rel_perc_diff = np.zeros(len(F_gurson))
    for i in range(0,len(F_gurson)):
        if F_gurson[i] <= threshold:
            rel_perc_diff[i] = rel_perc_diff[i-1]
        else:
            rel_perc_diff[i] = np.absolute((F_gurson[i]-F[i])/F_gurson[i]*100)
#     print(np.mean(rel_perc_diff))

interact(plot_data_and_potential_field_onlyyields, filename=filename_list, hof_index=hof_index_list,
         x_ind=range(5), y_ind=range(5), z_ind=range(5))

In [ ]:
def check_numerical_equivalency(filename, hof_index=0, sh=1, svm=1, vvf=1):
    data, data_labels, equation = _get_data_and_labels(filename, hof_index)
    
    equation_2 = equation.copy()
    equation_2._use_simplification = True
    equation_2._update()
    print('Bingo:',equation_2)
    eval_data = np.array([sh, svm, vvf])
    eval_data = eval_data.reshape(1,3)
    syield = 1
    gurson = (eval_data[:,1]/syield)**2+2*eval_data[:,2]*np.cosh(1.5*(eval_data[:,0]/syield))-(1+eval_data[:,2]**2)
    print('Gurson: (X_1)**2+2*X_2*cosh(1.5*(X_0))-(1+X_2**2)')
    equ_eval = equation.evaluate_equation_at(eval_data) # -0.8993549168808034
    print('Gurson Result:',gurson)
    print('Bingo Result:',equ_eval)
    
interact(check_numerical_equivalency, filename=filename_list, hof_index=hof_index_list,
         sh=(0.5,1.6), svm=(0.1,0.9), vvf=(0,0.4))

In [ ]:
def plot_data_and_value(filename, hof_index, value="potential", x_ind=0, y_ind=1, z_ind=2):
    data, data_labels, equation, fit_func = _get_data_and_labels(filename, hof_index, fit_func=True)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    c = "k"
    if value == "potential":
        c = equation.evaluate_equation_at(data.x)
    if value == "residual":
        fit_func.training_data = data
        c = np.abs(fit_func._fitness_function.evaluate_fitness_vector(equation))
    
    sc = ax.scatter(data.x[:, x_ind], data.x[:, y_ind], data.x[:, z_ind], c=c)
    ax.set_xlabel(data_labels[x_ind])
    ax.set_ylabel(data_labels[y_ind])
    ax.set_zlabel(data_labels[z_ind])
    fig.colorbar(sc)
    
    plt.show()

interact(plot_data_and_value, filename=filename_list, hof_index=hof_index_list,
         value=["potential", "residual"], x_ind=range(5), y_ind=range(5), z_ind=range(5))

In [26]:
def check_normailization(filename):
    archipelago = load_parallel_archipelago_from_file(filename)
    data_x = archipelago.island._full_training_data.x
    data_dx = archipelago.island._full_training_data.dx_dt
    d1 = h5py.File('d29.hdf5','a') 
    d1.create_dataset('data', data = data_x)
    d1.create_dataset('data_dxdt', data = data_dx)

    print(stats.describe(data_dx))
    print(np.max(data_dx, axis=0) - np.min(data_dx, axis=0))


In [27]:
check_normailization(filename_list[-1])

DescribeResult(nobs=22, minmax=(array([-0.2199275 , -0.21963304]), array([0.21914258, 0.1737653 ])), mean=array([-0.01014787, -0.05918256]), variance=array([0.03018527, 0.01675408]), skewness=array([0.13516847, 0.3100299 ]), kurtosis=array([-1.6633827 , -1.26154605]))
[0.43907008 0.39339834]


In [25]:
filename_list[-1]

'fs1d29e10_even/pkl_files/ML_project_k35_6_97000.pkl'